In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
import pandas as pd

from copy import deepcopy
import sklearn

### Import Data

In [2]:
cols = ['class','age','menopause','tumor-size',
        'inv-nodes','node-caps','deg-malig',
        'breast','breast-quad','irradiat']

df = pd.read_csv('./datasets/breast-cancer.data',names = cols)

Make all columns categorical

In [3]:
for this_col in cols[1:]:
    df[this_col] = df[this_col].astype('category')

Get mapping dict from label names to categorical indices

In [4]:
map_dicts_list = []

for col in df.columns:
    labels = np.unique(df[col])
    map_dicts_list.append(dict([[val,i] for i,val in enumerate(labels)]))
    
map_dicts_v2 = dict([[att,this_dict] for att,this_dict in zip(df.columns,map_dicts_list)])

Get instances for each class

In [5]:
print('Instance of first class:', np.sum(df['class'] == 'no-recurrence-events'))
print('Instance of second class:', np.sum(df['class'] == 'recurrence-events'))

Instance of first class: 201
Instance of second class: 85


Make normal and anomalies partitions

In [6]:
df_oc = df[df['class'] == 'no-recurrence-events']
df_oc = df_oc.drop('class',axis = 1)

In [7]:
df_anom = df[df['class'] == 'recurrence-events']
df_anom = df_anom.drop('class',axis = 1)

Make OH versions

In [8]:
df_oh = pd.get_dummies(df,columns = cols[1:])

In [9]:
oh_cols = df_oh.columns[1:]

In [10]:
df_oc_oh = df_oh[df_oh['class'] == 'no-recurrence-events']
df_oc_oh = df_oc_oh.drop('class',axis = 1)

In [11]:
df_anom_oh = df_oh[df_oh['class'] == 'recurrence-events']
df_anom_oh = df_anom_oh.drop('class',axis = 1)

### Train Trees

In [12]:
clfs = []
xs = []
ys = []

for col in cols[1:]:
    
    this_X = df_oc_oh.loc[:,[f for f in df_oc_oh.columns if not f.startswith(col)]]
    this_y = df_oc.loc[:,col]
    
    xs.append(this_X)
    ys.append(this_y)
    
    this_clf = tree.DecisionTreeClassifier()
    this_clf.fit(this_X,this_y)
    clfs.append(this_clf)

In [13]:
#plt.figure(figsize = (15,15))
#my_tree = tree.plot_tree(clfs[0])

### Get initial weights

In [14]:
from sklearn.metrics import recall_score, roc_auc_score

Initial weights for layer 1

In [15]:
w1_init = []
w2_init = []

for i,clf in enumerate(clfs):
    w1_init.append(recall_score(ys[i],clf.predict(xs[i]),average=None))
    w2_init.append(clf.score(xs[i],ys[i]))
    
w2_init = np.array(w2_init)

In [16]:
w1_init

[array([1.        , 0.95238095, 0.93650794, 0.92957746, 0.85      ,
        0.8       ]),
 array([0.9893617 , 1.        , 0.97058824]),
 array([0.85714286, 0.85185185, 0.86956522, 0.85294118, 0.66666667,
        0.65714286, 0.66666667, 0.625     , 0.5       , 0.5       ,
        0.2       ]),
 array([1.        , 1.        , 0.66666667, 0.84210526, 1.        ,
        1.        ]),
 array([1.  , 1.  , 0.96]),
 array([0.96610169, 0.93137255, 0.85      ]),
 array([0.95145631, 0.76530612]),
 array([1.        , 0.93333333, 0.81690141, 0.77777778, 0.6       ]),
 array([1.        , 0.91891892])]

In [17]:
w2_init

array([0.91542289, 0.9800995 , 0.73134328, 0.9800995 , 0.99502488,
       0.92537313, 0.86069652, 0.85074627, 0.98507463])

### Define some network functions

In [18]:
def neuron_l1(x_prime,weights,indxs):
    
    my_res = np.zeros((len(x_prime)))

    for i,this_x_prime in enumerate(x_prime):
        
        if indxs[i] >= len(weights):
            indxs[i] -= 1
        this_x_wrong = np.delete(this_x_prime,indxs[i])
        w_wrong = np.delete(weights,indxs[i])
        my_res[i] = this_x_prime[indxs[i]]*weights[indxs[i]]-np.mean(this_x_wrong*w_wrong)
        
    return my_res

In [19]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

### Experiment 1: Normal Data

Encoder

In [20]:
dt_y_hat = []

for i, clf in enumerate(clfs):
    dt_y_hat.append(clf.predict_proba(xs[i]))

In [123]:
w1 = deepcopy(w1_init)
w2 = deepcopy(w2_init)

lr = 0.001
iterations = 100
lrw1 = 0.01

losses = []
accs = []

y_norm = np.ones(len(df_oc_oh))

for i in range(iterations):
    
    hl1_this = np.zeros((len(df_oc_oh),len(cols[1:])))
    
    for j,x_prime in enumerate(dt_y_hat):

        x_prime_prime = np.argmax(x_prime*w1[j])
        
        label = cols[1:][j]
        num_labels = [map_dicts_v2[label][f] for f in ys[j]]
        
        for k in np.unique(num_labels):
            
            if k >= x_prime.shape[1]:
                break
            
            indices = np.where(num_labels==k)
            this_probs = x_prime[indices]
            x_prime_this_this = x_prime[indices]

            grad_wj = np.mean(x_prime_this_this[:,k])
            grad_wl = np.mean(-(1/len(np.unique(num_labels)))*np.delete(x_prime_this_this,k,axis = 1),axis = 0)
    
            w1[j][k] = w1[j][k] - lrw1*grad_wj
            
            for this_indx in [f for f in np.arange(np.max(num_labels)) if f != k]:
                
                #print('\t\t',this_indx)
                if this_indx < k:
                    grad_indx = this_indx
                elif this_indx > k:
                    grad_indx = this_indx-1
                    
                w1[j][this_indx] = w1[j][grad_indx] - lrw1*grad_wl[grad_indx]
    
    this_loss = []
    
    for j,x_prime in enumerate(dt_y_hat):
        
        label = cols[1:][j]
        num_labels = [map_dicts_v2[label][f] for f in ys[j]]
        hl1_this[:,j] = neuron_l1(x_prime,w1[j],num_labels)
    
        this_loss.append(np.mean(hl1_this[:,j]))
    
    if i % 10 == 0 or i == iterations-1:
        print('Iteration {}: Loss = {:0.4f}'.format(i+1,np.mean(this_loss)))

Iteration 1: Loss = 0.7111
Iteration 11: Loss = 0.6451
Iteration 21: Loss = 0.5792
Iteration 31: Loss = 0.5132
Iteration 41: Loss = 0.4473
Iteration 51: Loss = 0.3813
Iteration 61: Loss = 0.3153
Iteration 71: Loss = 0.2494
Iteration 81: Loss = 0.1834
Iteration 91: Loss = 0.1174
Iteration 100: Loss = 0.0581


### Test

In [141]:
dt_norm = []

for i, clf in enumerate(clfs):
    dt_norm.append(clf.predict_proba(xs[i]))

In [150]:
hl1_this_all = np.zeros((len(df_oh_x),len(cols[1:])))
preds = []
predsv2 = []
for j,x_prime in enumerate(dt_norm):
        
        label = cols[1:][j]
        num_labels = [map_dicts_v2[label][f] for f in ys[j]]
        #preds.append(np.argmax(x_prime*w1_init[j],axis = 1))
        #predsv2.append(np.argmax(x_prime*w1[j],axis = 1))
        
        preds.append(np.mean(neuron_l1(x_prime,w1[j],num_labels)))
        predsv2.append((neuron_l1(x_prime,w1_init[j],num_labels)))
        #hl1_this_all[:,j] = neuron_l1(x_prime,w1[j],num_labels)
        
#this_y_hat_all = sigmoid(np.dot(hl1_this_all,w2_init))
#np.mean((this_y_hat_all >= 0.5)==y_all)

In [152]:
np.mean(preds)

0.058082042136220896

In [153]:
np.mean(predsv2)

0.7029285512671855

In [133]:
df_ = df_oh.drop('class',axis = 1)
y_all = 1 - np.array([map_dicts_v2['class'][f] for f in df_oh['class']])

In [134]:
xs_test = []
ys_test = []

for col in cols[1:]:
    
    this_X = df_oh_x.loc[:,[f for f in df_oh_x.columns if not f.startswith(col)]]
    this_y = df.loc[:,col]
    
    xs_test.append(this_X)
    ys_test.append(this_y)

In [135]:
dt_all = []

for i, clf in enumerate(clfs):
    dt_all.append(clf.predict_proba(xs_test[i]))

In [138]:
hl1_this_all = np.zeros((len(df_oh_x),len(cols[1:])))
preds = []
predsv2 = []
for j,x_prime in enumerate(dt_all):
        
        label = cols[1:][j]
        num_labels = [map_dicts_v2[label][f] for f in ys_test[j]]
        #preds.append(np.argmax(x_prime*w1_init[j],axis = 1))
        #predsv2.append(np.argmax(x_prime*w1[j],axis = 1))
        
        preds.append(neuron_l1(x_prime,w1[j],num_labels))
        predsv2.append(neuron_l1(x_prime,w1_init[j],num_labels))
        #hl1_this_all[:,j] = neuron_l1(x_prime,w1[j],num_labels)
        
#this_y_hat_all = sigmoid(np.dot(hl1_this_all,w2_init))
#np.mean((this_y_hat_all >= 0.5)==y_all)

In [140]:
for i,col in enumerate(cols[1:]):
    print(col)
    print(np.mean(preds[i]))
    print(np.mean(predsv2[i]))

age
0.3189852024444505
0.6262055823675542
menopause
0.0039048784000786495
0.84585736541306
tumor-size
0.05297200857209073
0.3679870857726783
inv-nodes
0.03408878632748287
0.7201509017298492
node-caps
-0.009259436469963427
0.8275874125874126
deg-malig
0.030535814137165083
0.5868566406452748
breast
0.03427624624118668
0.3754791688068883
breast-quad
-0.08987398331722898
-0.09090728976292357
irradiat
0.014222552332310671
0.7324702324702325


In [107]:
w1_init

[array([1.        , 0.95238095, 0.93650794, 0.92957746, 0.85      ,
        0.8       ]),
 array([0.9893617 , 1.        , 0.97058824]),
 array([0.85714286, 0.85185185, 0.86956522, 0.85294118, 0.66666667,
        0.65714286, 0.66666667, 0.625     , 0.5       , 0.5       ,
        0.2       ]),
 array([1.        , 1.        , 0.66666667, 0.84210526, 1.        ,
        1.        ]),
 array([1.  , 1.  , 0.96]),
 array([0.96610169, 0.93137255, 0.85      ]),
 array([0.95145631, 0.76530612]),
 array([1.        , 0.93333333, 0.81690141, 0.77777778, 0.6       ]),
 array([1.        , 0.91891892])]